In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [2]:
import preprocessing as pre
import os
import pandas as pd
import pandas as pd
import numpy as np
import preprocessing as pre
from models.transformer import AudioTransformer
from sklearn.model_selection import train_test_split

c:\Projects\AutoAudio\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
directory = "../data/"
file_paths = []
labels = []
for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        file_path = os.path.join(directory, filename)
        label = int(filename[0])
        file_paths.append(file_path)
        labels.append(str(label))

data = pd.DataFrame({"file_path": file_paths, "label": labels})

features = pre.aggregate_audio_features(data)

In [4]:
data.reset_index(drop=True, inplace=True)
features, audios = pre.aggregate_audio_features(data)
features.reset_index(drop=True, inplace=True)
audios.reset_index(drop=True, inplace=True)

labels = data["label"]

test_size = 0.2
indices = labels.index
train_indices, test_indices = train_test_split(
    indices, test_size=test_size, random_state=42, shuffle=True
)
audios_train = audios.loc[train_indices]
audios_test = audios.loc[test_indices]

In [5]:
unique = np.unique(labels)
n_unique = len(unique)

label2id = {}
id2label = {}
for i, label in enumerate(unique):
    label2id[label] = i
    id2label[i] = label
model = AudioTransformer(n_unique, label2id, id2label)

c:\Projects\AutoAudio\env\Lib\site-packages\transformers\configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.fit(audios_train, audios_test)

Parameter 'function'=<function AudioTransformer.encode_dataset.<locals>.preprocess_function at 0x000002414C891E40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 4/4 [00:00<00:00, 250.65 examples/s]
                                             
100%|██████████| 1/1 [00:05<00:00,  3.25s/it]

{'eval_loss': 1.0905935764312744, 'eval_accuracy': 0.25, 'eval_runtime': 0.4517, 'eval_samples_per_second': 8.855, 'eval_steps_per_second': 2.214, 'epoch': 1.0}


100%|██████████| 1/1 [00:07<00:00,  7.44s/it]

{'train_runtime': 7.4327, 'train_samples_per_second': 2.153, 'train_steps_per_second': 0.135, 'train_loss': 1.096879482269287, 'epoch': 1.0}


In [7]:
predictions = model.predict(audios_test)
predictions

array(['1', '1', '0', '1'], dtype='<U1')